## Brute-Force

In [ ]:
# complexity: O(n * m)
def search_brute_force(s, p): 
    len_s, len_p = len(s), len(p) # source length, pattern length 
    
    # [0,(n-1)-(m-1)] → [0,n-m]
    for i in range(len_s - len_p + 1): 
        k = 0
        # p[0...m-1]
        for j in range(len_p): 
            if s[i + k] != p[j]:
                break
            k += 1
        if k == len_p: 
            return i
    return -1 

s = "aaaaaabcbabcabcdabcdeabcdef" 
p = "abcdab"
result = search_brute_force(s, p)
print(f" source: {s}")
print(f" target: {p}")
print(f" result: {result}")      

## Rabin-Karp

In [ ]:
# complexity: O((n - m + 1) * m) → O((n - m + 1) + c*m) → O(n + m)  
def search_rabin_karp(s, p): 
    len_s, len_p = len(s), len(p) # source length, pattern length 
    if len_s < len_p:
        return -1 
    if len_p == 0: 
        return 0
    B = 256     # base
    M = 1e9 + 7 # ver big prime number for mod
    # hash for pattern p 
    hash_p = 0  
    for i in range(len_p):
        hash_p = (hash_p * B + ord(p[i])) % M 
    # B^(m-1)
    BP = 1
    for i in range(len_p - 1): 
        BP = BP * B % M  
    # hashing of S[0, m-2]
    hash_s = 0
    for i in range(len_p - 1): 
        hash_s = (hash_s * B + ord(s[i])) % M
    # S[m-1,n-1]
    for i in range(len_p-1, len_s, 1): 
        # S[i-m+1...i]
        hash_s = (hash_s * B + ord(s[i])) % M
        if hash_s == hash_p: 
            # to handle hash collision 
            if search_equal(s, i-len_p+1, i, p):
                return i-len_p + 1
        # remove S[i-m+1] for hashing 
        hash_s = (hash_s - ord(s[i-len_p+1]) * BP % M + M) % M 
    return -1

"""
seach pattern p at S[l...r]
"""
def search_equal(s, l, r, p): 
    for i in range(l, r+1): 
        if s[i] != p[i-l]:
            return False
    return True 

s = "aaaaaabcbabcabcdabcdeabcdef" 
p = "abcdab"
result = search_rabin_karp(s, p)
print(f" source: {s}")
print(f" target: {p}")
print(f" result: {result}") 

## KMP

to visualize: [https://cmps-people.ok.ubc.ca/ylucet/DS/KnuthMorrisPratt.html](https://cmps-people.ok.ubc.ca/ylucet/DS/KnuthMorrisPratt.html)

In [ ]:
# complexity: O(n) ← O(2n + m)

def build_next_arr(p):
    next_arr = [-1] * len(p)
    k = -1
    next_arr[0] = -1
    for i in range(1, len(p)): 
        # refer to slides: next[i] -> next[i-1] (k) -> next[k] -> next[next[k]] -> ...
        while k != -1 and p[k + 1] != p[i]: 
            k = next_arr[k]
        if p[k + 1] == p[i]:
            k += 1
        next_arr[i] = k 
    return next_arr

def search_kmp(s, p): 
    len_s, len_p = len(s), len(p) # source length, pattern length 
    if len_s < len_p:
        return -1 
    if len_p == 0: 
        return 0
    next_arr = build_next_arr(p)
    j = 0
    for i in range(len_s): 
        while j > 0 and s[i] != p[j]: 
            # refer to slides: "bad char" found, look up next array  
            j = next_arr[j - 1] + 1 
        if s[i] == p[j]:
            j += 1
        if j == len_p:
            return i - len_p + 1 
    return -1

# test next array 
p = "bcbcbe"
print("   pattern: ", p)
print("next array: ", build_next_arr(p))
print()

s = "aaaaaabcbabcabcdabcdeabcdef" 
p = "abcdab"
result = search_kmp(s, p)
print(f" source: {s}")
print(f" target: {p}")
print(f" result: {result}") 

## BM

to visualize: [https://cmps-people.ok.ubc.ca/ylucet/DS/BoyerMoore.html](https://cmps-people.ok.ubc.ca/ylucet/DS/BoyerMoore.html)

wiki reference implementation: [https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_string-search_algorithm](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_string-search_algorithm)

In [ ]:
ALPHABET_SIZE = 256

"""
prebuilt bad char table, refer to slides 
"""
def build_badchar_tbl(p): 
    badchar_tbl = {}
    for j in range(ALPHABET_SIZE): 
        badchar_tbl[j] = -1
    for j in range(len(p)):
        badchar_tbl[ord(p[j])] = j
    return badchar_tbl 

"""
prebuilt suffix & prefix array, refer to slides 
"""
def build_goodsuffix_arr(p): 
    m = len(p)
    suffix = [-1 for _ in range(m)]
    prefix = [False for _ in range(m)]
    for i in range(m - 1): 
        # two pointers to compare suffix 
        j = i 
        k = 0
        while j >= 0 and p[j] == p[m - 1 - k]: 
            k += 1
            suffix[k] = j 
            j -= 1
        if j == -1: 
            prefix[k] = True
    return suffix, prefix 

"""
shift d for good suffix, refer to slides 

m: length of pattern 
j: index of bad char (P[j]) 
"""
def search_bm_shift_goodsuffix(suffix, prefix, m, j): 
    k =  m - 1 - j # length of good suffix 
    # look for right most suffix with length k 
    if suffix[k] != -1: 
        # "suffix" exists 
        return j - suffix[k] + 1
    # look for the largest prefix in P[j+2...m]: j+2 <= r <= m-1  
    for r in range(j+2, m, 1): 
        if prefix[m-r]: 
            # "prefix" exists 
            return r
    # no suffix, no prefix 
    return m 

def search_bm(s, p): 
    len_s, len_p = len(s), len(p) # source length, pattern length 
    if len_s < len_p:
        return -1 
    if len_p == 0: 
        return 0
    badchar_tbl = build_badchar_tbl(p)
    suffix, prefix = build_goodsuffix_arr(p)
    for i in range(len_s - len_p + 1): 
        j = len_p - 1 
        # comapre backward to find "bad char"
        while j >= 0 and s[i+j] == p[j]: 
            j -= 1
        if j < 0: 
            # p is found 
            return i 
        # bad char found at s[i+j] != p[j]
        # bad char shift d1
        d1 = j - badchar_tbl[ord(s[i+j])]
        d1 = 1 if d1 <= 0 else d1 
        # good suffix shift d2 
        d2 = 0 
        if len_p - 1 - j > 0: 
            d2 = search_bm_shift_goodsuffix(suffix, prefix, len_p, j)
        i += max(d1, d2)
    return -1
    
s = "aaaaaabcbabcabcdabcdeabcdef" 
p = "abcdab"
result = search_bm(s, p)
print(f" source: {s}")
print(f" target: {p}")
print(f" result: {result}") 

## Sunday

In [ ]:
def search_sunday(s, p): 
    len_s, len_p = len(s), len(p) # source length, pattern length 
    if len_s < len_p:
        return -1 
    if len_p == 0: 
        return 0
    # dictionary for shifting 
    badchar_dict = {}
    for i, val in enumerate(p): 
        # if val appears more than one time, badchar_dict only keeps the last one 
        badchar_dict[val] = len_p - i
    i = 0
    while i <= len_s - len_p:
        j = 0 
        while s[i+j] == p[j]: 
            j += 1
            if j == len_p: 
                return i
        # mismatch offset refer to slide 
        if i + len_p >= len_s:
            break; 
        offset = badchar_dict[s[i+len_p]] if s[i+len_p] in p else len_p + 1
        i += offset
    return -1

s = "aababcabcdabcdeabcdef" 
p = "abcdab"
result = search_sunday(s, p)
print(f" source: {s}")
print(f" target: {p}")
print(f" result: {result}") 

In [ ]:
# test performance 
import timeit
from random import randint
from IPython.display import HTML, display

def matching_test(func, s, p):  
    result = func(s,p) 

s = ""
p = "test"
n = 1000000
tests = 10
matching_funcs = ["search_brute_force", "search_rabin_karp", "search_kmp", "search_bm", "search_sunday"]
matching_setup = "from __main__ import s, p, matching_test, " + ",".join(matching_funcs)
matching_time = {}
matching_time["find"] = 0
for f in matching_funcs: 
    matching_time[f] = 0
for i in range(tests):
    # random string with length n 
    s = ''.join([chr(randint(97, 122)) for _ in range(n)])
    timer = timeit.Timer(stmt="s.find(p)", setup=matching_setup) 
    matching_time["find"] += timer.timeit(number=1)
    for f in matching_funcs:
        timer = timeit.Timer(stmt=f"matching_test({f}, s, p)", setup=matching_setup) 
        matching_time[f] += timer.timeit(number=1)
for k,v in matching_time.items():
    print("%s: %2.6fs" % (k.rjust(20), v/tests))
    

In [ ]:
# test similar string 
s = 'a' * 999999 + 'b'
p = 'a' * 99 + 'b'
matching_time = {}
matching_time["find"] = 0
for f in matching_funcs: 
    matching_time[f] = 0
timer = timeit.Timer(stmt="s.find(p)", setup=matching_setup) 
matching_time["find"] += timer.timeit(number=1)
for f in matching_funcs:
    timer = timeit.Timer(stmt=f"matching_test({f}, s, p)", setup=matching_setup) 
    matching_time[f] += timer.timeit(number=1)
for k,v in matching_time.items():
    print("%s: %2.6fs" % (k.rjust(20), v))

## Dynamic Programming

In [ ]:
# Fibonacci 
"""
Fibonacci sequence: 0,1,1,2,3,5,8, ...

when n = 1, fib(1) = 0
when n = 2, fib(2) = 1
when n > 2, fib(n) = fib(n-1) + fib(n-2)

Given a number N return the index value of Fibnonacci sequence  
"""
fib_run = 0 
fib_memo = {}

# Time Complexity: O(2^n)
def fib_recursive(n):
    global fib_run 
    fib_run += 1
    if (n == 1):
        return 0
    if (n == 2):
        return 1 
    return fib_recursive(n-1) + fib_recursive(n-2)

# Time Complexity: O(n)
def fib_recursive_memo(n): 
    global fib_run, fib_memo
    fib_run += 1
    # return directly from cache if found
    if (n in fib_memo): 
        return fib_memo[n]
    if (n == 1):
        return 0
    if (n == 2):
        return 1 
    # cache the result
    fib_memo[n] = fib_recursive_memo(n-1) + fib_recursive_memo(n-2)
    return fib_memo[n]

# Time Complexity: O(n)
def fib_dp(n): 
    memo = {}
    memo[1] = 0 
    memo[2] = 1
    for i in range(2, n+1, 1): 
        memo[i] = memo[i-1] + memo[i-2]
    return memo[n]

timer = timeit.Timer(stmt="fib_recursive(30)", setup="from __main__ import fib_recursive, fib_recursive_memo, fib_run, fib_memo") 
fib_run = 0 
fib_cache = {}
print("   n:", 30)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive(36)", setup="from __main__ import fib_recursive, fib_recursive_memo, fib_run, fib_memo") 
fib_run = 0 
fib_cache = {}
print("   n:", 36)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive(38)", setup="from __main__ import fib_recursive, fib_recursive_memo, fib_run, fib_memo") 
fib_run = 0 
fib_cache = {}
print("   n:", 38)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive_memo(38)", setup="from __main__ import fib_recursive, fib_recursive_memo, fib_run, fib_memo") 
fib_run = 0 
fib_cache = {}
print("!!! memo !!!")
print("   n:", 38)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))



In [ ]:
# 0-1 Knapsack Problem

# Returns the maximum value that
# can be put in a knapsack of
# capacity C
# time complexity: O(2^n)
# space complexity: O(1)
def knapsack1(w, v, n, C): 
    # base case 
    if n == 0 or C == 0:
        return 0
    # if weight of the nth item is more than C
    # then it cannot be included 
    if (w[n-1] > C): 
        return knapsack1(w, v, n-1, C)
    # return the max of two case: 
    # 1. nth item included 
    # 2. not included 
    else: 
        return max(v[n-1] + knapsack1(w, v, n-1, C - w[n-1]), knapsack1(w, v, n-1, C))

# time complexity: O(n * C)
# space complexity: O(n * C)
def knapsack2(w, v, n, C): 
    k = [[0 for x in range(C+1)] for x in range(n+1)]
    # build table k from bottom up 
    for i in range(n+1): 
        for c in range(C+1): 
            if i==0 or c==0: 
                k[i][c] = 0
            elif w[i-1] > c: 
                k[i][c] = k[i-1][c]
            else:
                k[i][c] = max(v[i-1] + k[i-1][c - w[i-1]], k[i-1][c])
    return k[n][C]

# time complexity: O(n * C)
# space complexity: O(2 * C)
def knapsack3(w, v, n, C): 
    # 2 rows only: i%2 
    k = [[0 for x in range(C+1)] for x in range(2)]
    # build table k from bottom up 
    for i in range(n+1): 
        for c in range(C+1): 
            if i==0 or c==0: 
                k[i % 2][c] = 0
            elif w[i-1] > c: 
                k[i % 2][c] = k[(i-1) % 2][c]
            else:
                k[i % 2][c] = max(v[i-1] + k[(i-1) % 2][c - w[i-1]], k[(i-1) % 2][c])
    return k[n % 2][C]

# time complexity: O(n * C)
# space complexity: O(C)
def knapsack4(w, v, n, C): 
    k = [0 for i in range(C+1)]
    for i in range(1, n+1):  
        # compute from the back (right to left) 
        for c in range(C, 0, -1):  
            if w[i-1] <= c:
                k[c] = max(v[i-1] + k[c - w[i-1]], k[c])
    return k[C]  
    


w = [10, 20, 30]
v = [60, 100, 120]
C = 50
n = len(v)
print("max value1:", knapsack1(w, v, n, C))
print("max value2:", knapsack2(w, v, n, C))
print("max value3:", knapsack3(w, v, n, C))
print("max value4:", knapsack4(w, v, n, C))

## Exercise 